# side model

In [1]:
import numpy as np
import pandas as pd

from jesse.helpers import date_to_timestamp

from strategies.BinanceBtcEntropyBarV1.config import SIDE

df_features = pd.read_parquet("data/features.parquet")

features = df_features[SIDE]
label = np.load("data/side_label.npy")
len_gap = len(label) - len(df_features)
label = label[len_gap:]

print(features.shape)
print(label.shape)

mask = features.index < date_to_timestamp("2024-09-30")
features_masked = features[mask]
label_masked = label[mask]

print(features_masked.shape)
print(label_masked.shape)
print(np.unique(label_masked, return_counts=True))

features.isna().sum().sort_values(ascending=False)

/opt/homebrew/Caskroom/miniconda/base/envs/jesse/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


(13219, 161)
(13219,)
(11485, 161)
(11485,)
(array([0, 1]), array([4536, 6949]))


voss                        0
roofing_filter_dt           0
vmd_win128_0                0
cwt_win1024_8               0
adx_7_dt                    0
                           ..
fisher                      0
forecast_oscillator_lag2    0
trendflex_ddt               0
cwt_win512_6                0
iqr_ratio_lag16             0
Length: 161, dtype: int64

In [2]:
import lightgbm as lgb

params = {
    "objective": "binary",
    "metric": "auc",
    "num_threads": -1,
    "verbose": -1,
    "is_unbalance": True,
    "extra_trees": False,
    "num_leaves": 100,
    "max_depth": 20,
    "min_gain_to_split": 1e-8,
    "min_data_in_leaf": 20,
    "lambda_l1": 1e-4,
    "lambda_l2": 1e-4,
}
dtrain = lgb.Dataset(features_masked, label_masked)
res = lgb.cv(params, dtrain, num_boost_round=100, nfold=5, stratified=True)
res.keys()

dict_keys(['valid auc-mean', 'valid auc-stdv'])

In [3]:
res["valid auc-mean"][-1]

0.9013170017639288

In [4]:
import lightgbm as lgb
import optuna


def objective(trial):
    METRIC = "auc"

    params = {
        "objective": "binary",
        "metric": METRIC,
        "num_threads": -1,
        "verbose": -1,
        "is_unbalance": trial.suggest_categorical("is_unbalance", [True, False]),
        "extra_trees": trial.suggest_categorical("extra_trees", [True, False]),
        "boosting": trial.suggest_categorical("boosting", ["gbdt", "dart"]),
        "num_leaves": trial.suggest_int("num_leaves", 31, 300),
        "max_depth": trial.suggest_int("max_depth", 30, 1000),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 1e-8, 1),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 500),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-4, 100),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-4, 100),
    }
    dtrain = lgb.Dataset(features, label)
    # dtest = lgb.Dataset(side_features_test, side_label_test)
    model_res = lgb.cv(
        params,
        dtrain,
        num_boost_round=trial.suggest_int("num_boost_round", 100, 1000),
        metrics=METRIC,
    )
    return model_res[f"valid {METRIC}-mean"][-1]


study = optuna.create_study(
    direction="maximize",
    pruner=optuna.pruners.HyperbandPruner(),
    sampler=optuna.samplers.TPESampler(),
)
study.optimize(objective, n_trials=100, n_jobs=1)

[I 2025-06-28 21:31:14,904] A new study created in memory with name: no-name-2b764a07-8757-46b2-928b-d006d8ce00ff
[I 2025-06-28 21:31:17,909] Trial 0 finished with value: 0.8923428036658724 and parameters: {'is_unbalance': True, 'extra_trees': True, 'boosting': 'gbdt', 'num_leaves': 173, 'max_depth': 744, 'min_gain_to_split': 0.3536648360841849, 'min_data_in_leaf': 186, 'lambda_l1': 75.04114647868786, 'lambda_l2': 11.78940154421232, 'num_boost_round': 829}. Best is trial 0 with value: 0.8923428036658724.
[I 2025-06-28 21:31:24,736] Trial 1 finished with value: 0.8954850373250434 and parameters: {'is_unbalance': True, 'extra_trees': True, 'boosting': 'dart', 'num_leaves': 257, 'max_depth': 53, 'min_gain_to_split': 0.9293523263197511, 'min_data_in_leaf': 176, 'lambda_l1': 37.40177378015296, 'lambda_l2': 17.931906746279335, 'num_boost_round': 378}. Best is trial 1 with value: 0.8954850373250434.
[I 2025-06-28 21:31:32,346] Trial 2 finished with value: 0.895329608599613 and parameters: {'i

In [5]:
print("side model long label: ")
study.best_params

side model long label: 


{'is_unbalance': False,
 'extra_trees': False,
 'boosting': 'gbdt',
 'num_leaves': 254,
 'max_depth': 33,
 'min_gain_to_split': 0.09915171296290792,
 'min_data_in_leaf': 102,
 'lambda_l1': 1.475684792664095,
 'lambda_l2': 16.28361140711989,
 'num_boost_round': 400}

In [6]:
import lightgbm as lgb  # noqa

params = {
    "objective": "binary",
    "num_threads": -1,
    "verbose": -1,
    **study.best_params,
}

dtrain = lgb.Dataset(features_masked, label_masked)
side_model = lgb.train(params, dtrain)
side_model_prod = lgb.train(params, lgb.Dataset(features, label))

In [7]:
import plotly.express as px

test_features = features[features.index > date_to_timestamp("2024-09-30")]
res = side_model.predict(test_features)

fig = px.histogram(res, nbins=100)
fig.show()

In [8]:
side_model.save_model("strategies/BinanceBtcEntropyBarV1/model/model_side.txt")
side_model_prod.save_model(
    "strategies/BinanceBtcEntropyBarV1/model/model_side_prod.txt"
)

# meta model

In [1]:
import numpy as np
import pandas as pd

from jesse.helpers import date_to_timestamp

from strategies.BinanceBtcEntropyBarV1.config import (
    META_ALL,
    SIDE,
    get_side_model,
)

df_features = pd.read_parquet("data/features.parquet")
meta_label = np.load("data/label_meta.npy")
print(f"{np.unique(meta_label, return_counts=True) = }")

side_model = get_side_model(False)

side_model_res = side_model.predict(df_features[SIDE])

df_features["model"] = side_model_res

meta_features = df_features[META_ALL]
print(meta_features.shape)
print(meta_label.shape)

mask = meta_features.index < date_to_timestamp("2025-01-01")
meta_features_masked = meta_features[mask]
meta_label_masked = meta_label[mask]

print(meta_features_masked.shape)
print(meta_label_masked.shape)
print(f"{np.unique(meta_label_masked, return_counts=True) = }")

meta_features.isna().sum(axis=0).sort_values(ascending=False)

/opt/homebrew/Caskroom/miniconda/base/envs/jesse/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


np.unique(meta_label, return_counts=True) = (array([0., 1.]), array([  719, 12500]))
(13219, 204)
(13219,)
(12115, 204)
(12115,)
np.unique(meta_label_masked, return_counts=True) = (array([0., 1.]), array([  652, 11463]))


sample_entropy_win512_array    0
fti_lag16                      0
fti_lag3                       0
acc_swing_index_lag12          0
price_variance_ratio_lag14     0
                              ..
hurst_coef_slow_lag17          0
price_variance_ratio_lag13     0
acc_swing_index_lag1           0
sample_entropy_win64_spot      0
model                          0
Length: 204, dtype: int64

In [5]:
import lightgbm as lgb
from sklearn.metrics import (
    f1_score,  # noqa
    fbeta_score,  # noqa
)

METRIC = "f1"


def eval_metric(preds, eval_dataset):
    metric_name = METRIC
    y_true = eval_dataset.get_label()
    value = f1_score(y_true, preds > 0.5, average="weighted")
    higher_better = True
    return metric_name, value, higher_better


params = {
    "objective": "binary",
    "num_threads": -1,
    "verbose": -1,
    "is_unbalance": True,
    "extra_trees": False,
    "num_leaves": 100,
    "max_depth": 20,
    "min_gain_to_split": 1e-8,
    "min_data_in_leaf": 20,
    "lambda_l1": 1e-4,
    "lambda_l2": 1e-4,
}
dtrain = lgb.Dataset(meta_features, meta_label)
res = lgb.cv(
    params, dtrain, num_boost_round=100, nfold=5, stratified=True, feval=eval_metric
)
res.keys()

dict_keys(['valid binary_logloss-mean', 'valid binary_logloss-stdv', 'valid f1-mean', 'valid f1-stdv'])

In [6]:
import optuna


def objective(trial):
    params = {
        "objective": "binary",
        "is_unbalance": trial.suggest_categorical("is_unbalance", [True, False]),
        "num_threads": -1,
        "verbose": -1,
        "extra_trees": trial.suggest_categorical("extra_trees", [True, False]),
        "boosting": trial.suggest_categorical("boosting", ["gbdt", "dart"]),
        "num_leaves": trial.suggest_int("num_leaves", 31, 500),
        "max_depth": trial.suggest_int("max_depth", 30, 1000),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 1e-8, 1),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 300),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 100),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 100),
    }
    dtrain = lgb.Dataset(meta_features, meta_label)
    # dtest = lgb.Dataset(meta_features_test, meta_label_test)
    model_res = lgb.cv(
        params,
        dtrain,
        num_boost_round=trial.suggest_int("num_boost_round", 100, 1500),
        stratified=True,
        feval=eval_metric,
    )
    return model_res[f"valid {METRIC}-mean"][-1]


study = optuna.create_study(
    direction="maximize",
    pruner=optuna.pruners.HyperbandPruner(),
    sampler=optuna.samplers.TPESampler(n_startup_trials=50),
)
study.optimize(objective, n_trials=200, n_jobs=1)

[I 2025-06-28 23:18:09,931] A new study created in memory with name: no-name-051dc284-93b8-40cf-a6c8-1305c4fc6d5a
[I 2025-06-28 23:18:41,237] Trial 0 finished with value: 0.8925924424531921 and parameters: {'is_unbalance': True, 'extra_trees': True, 'boosting': 'dart', 'num_leaves': 211, 'max_depth': 296, 'min_gain_to_split': 0.5863614930249749, 'min_data_in_leaf': 294, 'lambda_l1': 83.65656373882557, 'lambda_l2': 40.29213549966437, 'num_boost_round': 1120}. Best is trial 0 with value: 0.8925924424531921.
[I 2025-06-28 23:18:48,779] Trial 1 finished with value: 0.919173210651525 and parameters: {'is_unbalance': False, 'extra_trees': True, 'boosting': 'dart', 'num_leaves': 86, 'max_depth': 368, 'min_gain_to_split': 0.9920913733428581, 'min_data_in_leaf': 166, 'lambda_l1': 45.364105181130974, 'lambda_l2': 21.557208883010503, 'num_boost_round': 495}. Best is trial 1 with value: 0.919173210651525.
[I 2025-06-28 23:19:27,313] Trial 2 finished with value: 0.919173210651525 and parameters: {'

KeyboardInterrupt: 

In [7]:
print("meta model: ")
study.best_params

meta model: 


{'is_unbalance': True,
 'extra_trees': False,
 'boosting': 'dart',
 'num_leaves': 130,
 'max_depth': 140,
 'min_gain_to_split': 0.001051845042485327,
 'min_data_in_leaf': 177,
 'lambda_l1': 0.05431491352657014,
 'lambda_l2': 0.5123048433121706,
 'num_boost_round': 1068}

In [8]:
import lightgbm as lgb

params = {
    "objective": "binary",
    "num_threads": -1,
    "verbose": -1,
    **study.best_params,
}

model = lgb.train(
    params,
    lgb.Dataset(meta_features_masked, meta_label_masked),
)

prod_model = lgb.train(params, lgb.Dataset(meta_features, meta_label))

In [9]:
import plotly.express as px

res = model.predict(
    meta_features[meta_features.index > date_to_timestamp("2025-01-01")]
)
pred_label = (res > 0.5).astype(int)

fig = px.histogram(res, nbins=100)
fig.show()

In [10]:
model.save_model("strategies/BinanceBtcEntropyBarV1/model/model_meta.txt")
prod_model.save_model("strategies/BinanceBtcEntropyBarV1/model/model_meta_prod.txt")